In [ ]:
import os
from pathlib import Path
import pickle
from openai import OpenAI
from langfuse import Langfuse
from dotenv import load_dotenv
import json
from pprint import pprint

load_dotenv()

In [ ]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
langfuse = Langfuse(public_key=os.getenv("LANGFUSE_PUBLIC_KEY"), secret_key=os.getenv("LANGFUSE_SECRET_KEY"), host=os.getenv("LANGFUSE_HOST"))

cwd = Path(os.getcwd())
data_path = cwd.parent / "data"
documents_path = data_path / "pliki_z_fabryki"
facts_path = documents_path / "facts"
cache_path = documents_path / "cache"

In [ ]:
report_files = list(documents_path.glob("*.txt"))

reports = [
        {"file_name": report_file.name, "content": report_file.read_text()}
        for report_file in report_files
    ]

fact_files = list(facts_path.glob("*.txt"))

facts = [
    {"file_name": fact_file.name, "content": fact_file.read_text()}
    for fact_file in fact_files
]

## Extract simple keywords

No additional context knowledge

In [ ]:
def generate_simple_keywords_from_report(report_file_name: str, report_content: str) -> dict:
    """Generate keywords based only on report content without external knowledge."""
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    
    system_prompt = f"""
    Twoją rolą jest analiza raportów bezpieczeństwa fabryki.
    Zadanie polega na wybraniu słów kluczowych z raportu.
    Słowa kluczowe muszą być zawsze w mianowniku.
    W słowach kluczowych uwzględnij również numer sektora fabryki którego dotyczy raport. Informacja ta znajduje się w sekcji ze źródłem raportu.
    Zamieniając sektor na słowo kluczowe zastosuj zawsze format: "Sektor C1", "Sektor B2" i podobne.
    Każdy raport musi mieć informację o sektorze fabryki w słowach kluczowych.
    Format wyjściowy powinien być w postaci listy słów kluczowych oddzielonych przecinkami. Nie znaków końca linii.
    """

    user_prompt = f"""
    <źródło raportu>{report_file_name}</źródło raportu>
    <zawartość raportu>{report_content}</zawartość raportu>
    """

    result = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
    )

    keywords = result.choices[0].message.content
    keywords = keywords.replace("\n", "")
    keywords = keywords.split(", ")
    keywords = [keyword.strip() for keyword in keywords]

    data = {
        "report_file_name": report_file_name,
        "content": report_content,
        "keywords": keywords
    }

    return data

In [ ]:
reports_with_keywords = [
    generate_simple_keywords_from_report(report["file_name"], report["content"])
    for report in reports
]

In [ ]:
def generate_simple_keywords_from_facts(facts_file_name: str, facts_content: str) -> dict:
    """Generate keywords based only on facts."""
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    
    system_prompt = f"""
    Twoją rolą jest analiza faktów dostarczonych w formie notatki.
    Zadanie polega na wybraniu słów kluczowych.
    Słowa kluczowe muszą być zawsze w mianowniku.
    Format wyjściowy powinien być w postaci listy słów kluczowych oddzielonych przecinkami. Nie znaków końca linii.
    """

    user_prompt = facts_content

    result = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
    )

    keywords = result.choices[0].message.content
    keywords = keywords.replace("\n", "")
    keywords = keywords.split(", ")
    keywords = [keyword.strip() for keyword in keywords]

    data = {
        "facts_file_name": facts_file_name,
        "facts_content": facts_content,
        "keywords": keywords
    }

    return data

In [ ]:
facts_with_keywords = [
    generate_simple_keywords_from_facts(fact["file_name"], fact["content"])
    for fact in facts
]

## Greedy matching

In [ ]:
for report in reports_with_keywords:
    context_adjusted_keywords = []
    for rpt_keyword in report["keywords"]:
        context_adjusted_keywords.append(rpt_keyword)
        for fact in facts_with_keywords:
            if rpt_keyword in fact["keywords"]:
                for fact_keyword in fact["keywords"]:
                    if fact_keyword not in context_adjusted_keywords:
                        context_adjusted_keywords.append(fact_keyword)

    report["context_adjusted_keywords"] = context_adjusted_keywords

## Test

In [ ]:
from aidevs3.poligon import send

load_dotenv()

key = os.environ.get("AG3NTS_API_KEY")
url = f"{os.environ.get("AG3NTS_CENTRALA_URL")}/report"

result = {
    report["report_file_name"]: ", ".join(report["context_adjusted_keywords"])
    for report in reports_with_keywords
}

print(json.dumps(result))

res = send(url, answer=result, apikey=key, task="dokumenty")
print(res)